## Stochastic MPC using State Estimation and Terminal Cost

# Specify perturbation

In [1]:
import numpy as np
import gstools as gs
import scipy.stats


# np.random.seed(42)
nx = 25  # number of grid blocks
Ne = 10 # number of perturbation
bmean = 5.0
b_std = bmean/2
bmin = 3.0
bmax = 8.0

if False:
    bw = np.zeros((nx, Ne))
    
    for i in range(Ne):
        model = gs.Gaussian(dim=1, var=0.5, len_scale=5)
        srf = gs.SRF(model)
        x = range(25)
        srf(x, mesh_type='structured')
        fieldcdf = scipy.stats.norm.cdf(srf.field, 0, 1)
        a = (bmin - bmean) / b_std
        b = (bmax - bmean) / b_std
        var = scipy.stats.truncnorm.ppf(fieldcdf, a, b)
        _bw = var*b_std + bmean

        bw[:,i] = _bw

    np.save("./data/bw.npy", bw)
    
if False: 
    model = gs.Gaussian(dim=1, var=0.5, len_scale=5)
    srf = gs.SRF(model)
    x = range(25)
    srf(x, mesh_type='structured')
    fieldcdf = scipy.stats.norm.cdf(srf.field, 0, 1)
    a = (bmin - bmean) / b_std
    b = (bmax - bmean) / b_std
    var = scipy.stats.truncnorm.ppf(fieldcdf, a, b)
    _bw = var*b_std + bmean

    b_true = _bw*1
    
    np.save("./data/b_true.npy", b_true)
    



## All simulation function goes here

In [2]:

import casadi as ca
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from setup_sbl_te import setup_sbl_ocp, BLParamsSmpcShort
from solver_socp_te import SolverOcp

def make_ensemble_propagator(f_discrete, nx, Ne):
    Sw_flat = ca.SX.sym("Sw_flat", nx*Ne)
    u_sym = ca.SX.sym("u")
    aw_sym = ca.SX.sym("aw", nx, Ne)
    bw_sym = ca.SX.sym("bw", nx, Ne)
    outs = []
    for i in range(Ne):
        Si = Sw_flat[i*nx:(i+1)*nx]
        outs.append(f_discrete(Si, u_sym, aw_sym[:,i], bw_sym[:,i]))
        # outs.append(f_discrete(Si, u_sym, aw_sym[i], bw_sym[i]))
    Sw_next_flat = ca.vertcat(*outs)
    return ca.Function("propagate_ensemble", [Sw_flat, u_sym, aw_sym, bw_sym], [Sw_next_flat])


def run_socp_problem(n_mpc_steps, N_mpc, nx, bw, suffix:str):
    
    params_mpc = BLParamsSmpcShort(N_mpc = N_mpc, 
                                Ne = Ne,
                                nx = nx,
                                bw = bw)

    qocp = np.ones(params_mpc.Nt)
    nx = params_mpc.nx

    qt = np.array([1.0]*params_mpc.N_mpc)  # initial guess for control

    u_traj = qt*1

    _Sw0 = [1.0]+[0.2]*(params_mpc.nx-1)
    Sw0 = np.array(_Sw0*params_mpc.Ne)  # initial state
    Sw_mpc = Sw0.copy()

    qmpc = [1.0]

    for mpc_step in tqdm(range(n_mpc_steps)):
        _qocp = qocp[(N_mpc+mpc_step):]
        
        ocp = setup_sbl_ocp(params_mpc, 
                            qmpc=qmpc,
                            qocp=_qocp)
        
        aw = params_mpc.aw
        bw = params_mpc.bw
        
        if mpc_step == 0:
            f_discrete = ocp.bl.simulate_at_k
            f = make_ensemble_propagator(f_discrete, nx, Ne)
        
        solver = SolverOcp(ocp, itk=mpc_step)
        solver.set_initial_guess(Sw0, qt)
        x_traj, u_traj = solver.solve(Sw0)

    qocp = np.insert(u_traj[:,0], 0, 1.0)[:-1]
    # Compute the cost from the OCP solutions
    Sw_traj = []
    cost_traj = []
    for k, q in enumerate(qocp):
        
        costs = []
        for i in range(params_mpc.Ne):
            Sw = Sw_mpc[i*nx:(i+1)*nx]
            cost = -ocp.bl._stage_cost(Sw, q, aw, bw[-1,i])*(0.99**(k*0.03))
            costs.append(cost)
            
        Sw_mpc = f(Sw_mpc, q, aw, bw)
        Sw_traj.append(Sw_mpc)
        cost_traj.append(costs)
        

    qocp = np.array(qocp)
    Sw_traj = np.array(Sw_traj)  # (n_mpc_steps, nx*Ne)
    cost_traj = np.array(cost_traj)  # (n_mpc_steps, Ne)

    np.save(f"./data/{suffix}_qocp.npy", qocp)
    np.save(f"./data/{suffix}_Sw_traj.npy", Sw_traj)
    np.save(f"./data/{suffix}_cost_traj.npy", cost_traj)
        
def run_smpc_problem(n_mpc_steps, N_mpc, nx, bw, b_true, suffix:str, qterm, Sw_wbt, N_obs, sigma):
    import casadi as ca
    import numpy as np
    import matplotlib.pyplot as plt
    from tqdm.notebook import tqdm

    from setup_sbl_te import setup_sbl_ocp, setup_sbl_ocp_without_terminal, BLParamsSmpcShort, BLParamsTrue
    from solver_socp_te import SolverOcp

    from kalman_filter import ESMDA
    
    """
    
    qterm : array -> used for temrinal cost
            "constant" -> used the last control found in the solver
            None -> no terminal cost
    """
        
    params_mpc = BLParamsSmpcShort(N_mpc = N_mpc, 
                                Ne = Ne,
                                nx = nx,
                                bw = bw)

    params_mpc_true = BLParamsTrue(N_mpc = N_mpc, 
                                Ne = 1,
                                nx = nx,
                                bw = b_true[:,np.newaxis])

    aw = params_mpc.aw
    esmda = ESMDA(m=bw, g_func=None, g_obs=np.array([0.01]), alphas=[], cd=[0.01])

    qt = np.array([1.0]*params_mpc.N_mpc)  # initial guess for control

    u_traj = qt*1

    _Sw0 = [1.0]+[0.2]*(params_mpc.nx-1)
    Sw0 = np.array(_Sw0*params_mpc.Ne)  # initial state
    Sw_mpc = Sw0.copy()
    Sw_true = np.array(_Sw0.copy())

    qmpc = [1.0]

    cost_traj = []

    Sw_traj = []
    Sw_true_traj = []

    bw_traj = []
    
    n_obs_data_count = 0
    
    is_breakthrough = False
    g_meas = np.zeros((Ne, N_obs))
    g_obs = np.zeros((N_obs))
    
    for mpc_step in tqdm(range(n_mpc_steps)):
        
        
        params_mpc = BLParamsSmpcShort(N_mpc = N_mpc, 
                                Ne = Ne,
                                nx = nx,
                                bw = bw)
        
        if qterm is None:
            _qocp = []
            ocp = setup_sbl_ocp_without_terminal(params_mpc, 
                                qmpc=qmpc,
                                qocp=_qocp)
        
        elif qterm is "constant":
            
            if mpc_step == 0:
                qtc = np.array([1.0]*(n_mpc_steps+1))
            else:
                qtc = np.array([u_traj[-1,0]]*(n_mpc_steps+1)) 
            
            _qocp = qtc[(N_mpc+mpc_step+1):]
            ocp = setup_sbl_ocp(params_mpc, 
                                qmpc=qmpc,
                                qocp=_qocp)
            
        else:
            _qocp = qterm[(N_mpc+mpc_step+1):]
            ocp = setup_sbl_ocp(params_mpc, 
                                qmpc=qmpc,
                                qocp=_qocp)
        
        bw = params_mpc.bw
        b_true = params_mpc_true.bw
        
        if mpc_step == 0:
            f_discrete = ocp.bl.simulate_at_k
            f = make_ensemble_propagator(f_discrete, nx, Ne)
        
        # activate parameter estimation procedure here
        # only when water breakthrough is observed, i.e., saturation at production well > 0.5
        # if Sw_true[-1] > 0.5 for any i in range(Ne)
                
        if Sw_true[-1] > Sw_wbt:
            
            if not is_breakthrough:
                t_breakthrough = mpc_step*1
                is_breakthrough = True
            
            # collect ensemble response
            for i in range(Ne):
                Sw_last = Sw_mpc[(i+1)*nx-1]
                g_meas[i, n_obs_data_count] = Sw_last
                    
            # collect observed response
            g_obs[n_obs_data_count] = Sw_true[-1]
                    
            if n_obs_data_count < N_obs-1:
                n_obs_data_count += 1
                
            else:
                # perform parameter estimation update here
        
                # cd = np.array([0.00000001]*N_obs)
                cd = np.array([sigma]*N_obs)
                # cd = np.array([0.000005]*N_obs)
                
                bw_post = esmda.update(bw.T, g_meas, g_obs, 1, cd)
                bw = bw_post.T
                
                bw_traj.append(bw)
        
                # reset 
                g_meas = np.zeros((Ne, N_obs))
                g_obs = np.zeros((N_obs))
                n_obs_data_count = 0
        
        costs = []
        for i in range(params_mpc.Ne):
            Sw = Sw_mpc[i*nx:(i+1)*nx]
            cost = -ocp.bl._stage_cost(Sw, qmpc[-1], aw, bw[-1,i])*(0.99**(mpc_step*0.03))
            costs.append(cost)
        cost_traj.append(costs)
        
        # simulate the ensemble
        Sw_mpc = f(Sw_mpc, qmpc[-1], aw, bw).full()[:,0]

        # simulate the true model
        Sw_true = f_discrete(Sw_true, qmpc[-1], aw, b_true[:,0]).full()[:,0]
        
        solver = SolverOcp(ocp, itk=mpc_step)
        
        # warm start
        if mpc_step == 0:
            solver.set_initial_guess(Sw0, qt)
        else:
            _qt = np.insert(u_traj[1:,0], N_mpc-1, u_traj[-1,0])
            solver.set_initial_guess(Sw0, _qt)
            
        x_traj, u_traj = solver.solve(Sw0)
        
        
        qmpc.append(u_traj[0,0]*1)
        
        Sw_traj.append(Sw_mpc)
        Sw_true_traj.append(Sw_true)
        
        
    cost_traj = np.array(cost_traj)
    
    np.save(f"./data/{suffix}_bw_traj.npy", bw_traj)
    np.save(f"./data/{suffix}_qmpc.npy", qmpc)
    np.save(f"./data/{suffix}_Sw_traj.npy", Sw_traj)
    np.save(f"./data/{suffix}_cost_traj.npy", cost_traj)

def get_flow_rate(Sw_traj, qt, Ne, aw, bw):
    from buckley_leverette_te import BuckleyLeverette
    bl = BuckleyLeverette()
    fw_func = bl.fractional_flow

    
    Nt = qt.shape[0]-1
    Ne = 10

    qwt = []
    qot = []
    swt = []
    fwt = []
    costt = []
    for t in range(Nt):
        Sw_flat = Sw_traj[t,:]
        
        qws = []
        qos = []
        sws = []
        fws = []
        costs = []
        for i in range(Ne):
            Sw = Sw_flat[(i+1)*nx-1]
            _b = bw[-1,i]
            
            fw = fw_func(Sw, 100.0, _b)
            qw = fw*qt[t]
            qo = (1-fw)*(qt[t])
            
            qws.append(qw.full()[0,0])
            qos.append(qo.full()[0,0])
            sws.append(Sw)
            fws.append(fw)

            cost = bl._stage_cost([Sw], qt[t], aw, _b)*(0.99**(t*0.03))
            
            costs.append(cost)

        qwt.append(qws)
        qot.append(qos)
        swt.append(sws)
        costt.append(costs)
        fwt.append(fws)
        
    fwt = np.array(fwt)
    swt = np.array(swt)
    costt = np.array(costt)

    return qwt, qot, swt, costt, fwt

# def calculate_cost(f_discrete, Swinit, qseq, aw, bw, Ne, nx):
#     """Calculate cost function

#     Args:
#         Swinit (_type_): Initial Sw, shape (nx, Ne)
#         qseq (_type_): Control sequence, shape (Nt)
#         aw: a uncertainty, shape (nx, Ne)
#         bw: b uncertainty, shape (nx, Ne)
#     """
    
#     Sw = Swinit*1
    
#     cost_traj = []
#     for k, q in enumerate(qseq):
#         costs = []
#         new_Sw = []
#         for i in range(Ne):
#             _Sw = Sw[i*nx:(i+1)*nx]
#             cost = -ocp.bl._stage_cost(_Sw, q, aw, bw[-1,i])*(0.99**(k*0.03))
#             # cost = -ocp.bl._stage_cost(_Sw, q)*(1.0**k)
#             costs.append(cost.full()[0,0])
            
#             _Sw = f_discrete(_Sw, q, aw, bw[:,i]).full()[:,0]
#             new_Sw.extend(_Sw)
            
#         cost_traj.append(costs)
#         Sw = np.array(new_Sw)
        
#     return np.array(cost_traj)



<>:155: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
<>:155: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
/tmp/ipykernel_244393/3900836416.py:155: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
  elif qterm is "constant":


## Run different simulations

In [3]:


## Run SOCP
if False:
    n_mpc_steps = 1
    N_mpc = 75
    nx = 25
    bw = np.load("./data/bw.npy")
    run_socp_problem(n_mpc_steps=n_mpc_steps, 
                     N_mpc=N_mpc, 
                     nx = nx,
                     bw = bw,
                     suffix="socp")
    
## Run SMPC, WITHOUT model update, constant control for terminal
if False:
    n_mpc_steps = 74
    N_mpc = 5
    nx = 25
    qterm = "constant"
    bw = np.load("./data/bw.npy")
    b_true = np.load("./data/b_true.npy")
    Sw_wbt = 2.0 # with model update 
    N_obs = 5 # number of data points to be used simultaneously in the parm estimation
    sigma = 0.01
        
    run_smpc_problem(n_mpc_steps=n_mpc_steps,
                     N_mpc=N_mpc,
                     nx=nx,
                     bw=bw,
                     b_true=b_true,
                     suffix="nse_cc",
                     qterm=qterm,
                     Sw_wbt=Sw_wbt,
                     N_obs=N_obs,
                     sigma=sigma
                    )
    

## Run SMPC, WITH model update
if False:
    n_mpc_steps = 74
    N_mpc = 5
    nx = 25
    qterm = np.load("./data/socp_qocp.npy")
    bw = np.load("./data/bw.npy")
    b_true = np.load("./data/b_true.npy")
    Sw_wbt = 0.2 # with model update 
    N_obs = 10 # number of data points to be used simultaneously in the parm estimation 
    sigma = 0.000001
        
    run_smpc_problem(n_mpc_steps=n_mpc_steps,
                     N_mpc=N_mpc,
                     nx=nx,     
                     bw=bw,
                     b_true=b_true,
                     suffix="wse",
                     qterm=qterm,
                     Sw_wbt=Sw_wbt,
                     N_obs=N_obs,
                     sigma=sigma
                    )
    
if False:
    n_mpc_steps = 74
    N_mpc = 5
    nx = 25
    qterm = np.load("./data/socp_qocp.npy")
    bw = np.load("./data/bw.npy")
    b_true = np.load("./data/b_true.npy")
    Sw_wbt = 2.0 # WITHOUT model update 
    N_obs = 10 # number of data points to be used simultaneously in the parm estimation
    sigma = 0.1
        
    run_smpc_problem(n_mpc_steps=n_mpc_steps,
                     N_mpc=N_mpc,
                     nx=nx,
                     bw=bw,
                     b_true=b_true,
                     suffix="nse",
                     qterm=qterm,
                     Sw_wbt=Sw_wbt,
                     N_obs=N_obs,
                     sigma=sigma
                    )
    
    
## Run SMPC without terminal cost

if False: 
    for N_mpc in [5, 10, 20]:
        n_mpc_steps = 74
        nx = 25
        qterm = None # <- identifier to run optimization without terminal cost
        bw = np.load("./data/bw.npy")
        b_true = np.load("./data/b_true.npy")
        Sw_wbt = 1.0 
        N_obs = 10 # number of data points to be used simultaneously in the parm estimation
        sigma = 0.001
            
        run_smpc_problem(n_mpc_steps=n_mpc_steps,
                        N_mpc=N_mpc,
                        nx=nx,
                        bw=bw,
                        b_true=b_true,
                        suffix=f"notc_{N_mpc}",
                        qterm=qterm,
                        Sw_wbt=Sw_wbt,
                        N_obs=N_obs,
                        sigma=sigma
                        )
    
        
        

    
    




In [4]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


## More runs

1. Run 10 different initial ensemble. Compare the solution from SOCP, SMPC without model update, and with model update.
2. Run 1 "bad" initial ensemble. Bad initial ensemble = little to no coverage of the true parameters

In [ ]:
import numpy as np
import gstools as gs
import scipy.stats


# np.random.seed(42)
nx = 25  # number of grid blocks
Ne = 10 # number of perturbation
bmean = 5.0
b_std = bmean/2
bmin = 3.0
bmax = 8.0

N_perturbations = 10

for kk in range(N_perturbations):
    print(f"----------------------------------PERTURBATION NO: {kk} --------------------------------------")

    if True:
        bw = np.zeros((nx, Ne))
        
        for i in range(Ne):
            model = gs.Gaussian(dim=1, var=0.5, len_scale=5)
            srf = gs.SRF(model)
            x = range(25)
            srf(x, mesh_type='structured')
            fieldcdf = scipy.stats.norm.cdf(srf.field, 0, 1)
            a = (bmin - bmean) / b_std
            b = (bmax - bmean) / b_std
            var = scipy.stats.truncnorm.ppf(fieldcdf, a, b)
            _bw = var*b_std + bmean

            bw[:,i] = _bw

        np.save(f"./data/p{kk}_bw.npy", bw)

    if True:
        n_mpc_steps = 1
        N_mpc = 75
        nx = 25
        bw = np.load(f"./data/p{kk}_bw.npy")
        run_socp_problem(n_mpc_steps=n_mpc_steps, 
                        N_mpc=N_mpc, 
                        nx = nx,
                        bw = bw,
                        suffix=f"p{kk}_socp")

    ## Run SMPC, WITH model update
    if True:
        n_mpc_steps = 74
        N_mpc = 5
        nx = 25
        qterm = np.load(f"./data/p{kk}_socp_qocp.npy")
        bw = np.load(f"./data/p{kk}_bw.npy")
        b_true = np.load("./data/b_true.npy")
        Sw_wbt = 0.2 # with model update 
        N_obs = 5 # number of data points to be used simultaneously in the parm estimation 
        sigma = 0.1
            
        run_smpc_problem(n_mpc_steps=n_mpc_steps,
                        N_mpc=N_mpc,
                        nx=nx,     
                        bw=bw,
                        b_true=b_true,
                        suffix=f"p{kk}_wse",
                        qterm=qterm,
                        Sw_wbt=Sw_wbt,
                        N_obs=N_obs,
                        sigma=sigma
                        )
        
    if True:
        n_mpc_steps = 74
        N_mpc = 5
        nx = 25
        qterm = np.load(f"./data/p{kk}_socp_qocp.npy")
        bw = np.load(f"./data/p{kk}_bw.npy")
        b_true = np.load("./data/b_true.npy")
        Sw_wbt = 2.0 # WITHOUT model update 
        N_obs = 10 # number of data points to be used simultaneously in the parm estimation
        sigma = 0.1
            
        run_smpc_problem(n_mpc_steps=n_mpc_steps,
                        N_mpc=N_mpc,
                        nx=nx,
                        bw=bw,
                        b_true=b_true,
                        suffix=f"p{kk}_nse",
                        qterm=qterm,
                        Sw_wbt=Sw_wbt,
                        N_obs=N_obs,
                        sigma=sigma
                        )

----------------------------------PERTURBATION NO: 0 --------------------------------------


  0%|          | 0/1 [00:00<?, ?it/s]

Control sequence used in cost calculation: [1.0, SX(q_0), SX(q_1), SX(q_2), SX(q_3), SX(q_4), SX(q_5), SX(q_6), SX(q_7), SX(q_8), SX(q_9), SX(q_10), SX(q_11), SX(q_12), SX(q_13), SX(q_14), SX(q_15), SX(q_16), SX(q_17), SX(q_18), SX(q_19), SX(q_20), SX(q_21), SX(q_22), SX(q_23), SX(q_24), SX(q_25), SX(q_26), SX(q_27), SX(q_28), SX(q_29), SX(q_30), SX(q_31), SX(q_32), SX(q_33), SX(q_34), SX(q_35), SX(q_36), SX(q_37), SX(q_38), SX(q_39), SX(q_40), SX(q_41), SX(q_42), SX(q_43), SX(q_44), SX(q_45), SX(q_46), SX(q_47), SX(q_48), SX(q_49), SX(q_50), SX(q_51), SX(q_52), SX(q_53), SX(q_54), SX(q_55), SX(q_56), SX(q_57), SX(q_58), SX(q_59), SX(q_60), SX(q_61), SX(q_62), SX(q_63), SX(q_64), SX(q_65), SX(q_66), SX(q_67), SX(q_68), SX(q_69), SX(q_70), SX(q_71), SX(q_72), SX(q_73)]
Control sequence used in cost calculation: [1.0, SX(q_0), SX(q_1), SX(q_2), SX(q_3), SX(q_4), SX(q_5), SX(q_6), SX(q_7), SX(q_8), SX(q_9), SX(q_10), SX(q_11), SX(q_12), SX(q_13), SX(q_14), SX(q_15), SX(q_16), SX(q_17), SX

In [ ]:
## Run SMPC, WITH model update
if False:
    n_mpc_steps = 74
    N_mpc = 5
    nx = 25
    qterm = np.load("./data/socp_qocp.npy")
    bw = np.load("./data/bw_bad.npy")
    b_true = np.load("./data/b_true.npy")
    Sw_wbt = 0.2 # with model update 
    N_obs = 5 # number of data points to be used simultaneously in the parm estimation 
    sigma = 0.1
        
    run_smpc_problem(n_mpc_steps=n_mpc_steps,
                     N_mpc=N_mpc,
                     nx=nx,     
                     bw=bw,
                     b_true=b_true,
                     suffix="bad_wse",
                     qterm=qterm,
                     Sw_wbt=Sw_wbt,
                     N_obs=N_obs,
                     sigma=sigma
                    )
    
if False:
    n_mpc_steps = 74
    N_mpc = 5
    nx = 25
    qterm = np.load("./data/socp_qocp.npy")
    bw = np.load("./data/bw_bad.npy")
    b_true = np.load("./data/b_true.npy")
    Sw_wbt = 2.0 # WITHOUT model update 
    N_obs = 10 # number of data points to be used simultaneously in the parm estimation
    sigma = 0.1
        
    run_smpc_problem(n_mpc_steps=n_mpc_steps,
                     N_mpc=N_mpc,
                     nx=nx,
                     bw=bw,
                     b_true=b_true,
                     suffix="bad_nse",
                     qterm=qterm,
                     Sw_wbt=Sw_wbt,
                     N_obs=N_obs,
                     sigma=sigma
                    )

  0%|          | 0/74 [00:00<?, ?it/s]

Control sequence used in cost calculation: [1.0, SX(q_0), SX(q_1), SX(q_2), SX(q_3), SX(q_4), np.float64(0.9409427142484854), np.float64(0.9389287130368776), np.float64(0.9371604075179969), np.float64(0.9355801002987734), np.float64(0.9341395664952877), np.float64(0.9328169330297097), np.float64(0.9315887683192959), np.float64(0.9304354008080462), np.float64(0.9293503066660387), np.float64(0.9283206254127375), np.float64(0.9273397538720493), np.float64(0.9264052085049441), np.float64(0.925511147482451), np.float64(0.9246550333466057), np.float64(0.9238364316210416), np.float64(0.9230517160308442), np.float64(0.9222994500608251), np.float64(0.9215792900162303), np.float64(0.9208870948338501), np.float64(0.9202234408552877), np.float64(0.9195843191501615), np.float64(0.9189695393455907), np.float64(0.9183761159758055), np.float64(0.9178035576283241), np.float64(0.9172497353082315), np.float64(0.9167142162918668), np.float64(0.9161954832620502), np.float64(0.9156933604003569), np.float64(

# Compute cost using the controls obtain from various method on the true model

In [7]:

# from buckley_leverette_te import BuckleyLeverette
# from setup_sbl_te import setup_sbl_ocp, BLParamsSmpcShort, BLParamsTrue

# def calculate_cost(f_discrete, Swinit, qseq, aw, bw, Ne, nx):
#     """Calculate cost function

#     Args:
#         Swinit (_type_): Initial Sw, shape (nx, Ne)
#         qseq (_type_): Control sequence, shape (Nt)
#         aw: a uncertainty, shape (nx, Ne)
#         bw: b uncertainty, shape (nx, Ne)
#     """
    
#     Sw = Swinit*1
    
#     cost_traj = []
#     for k, q in enumerate(qseq):
#         costs = []
#         new_Sw = []
#         for i in range(Ne):
#             _Sw = Sw[i*nx:(i+1)*nx]
#             cost = -ocp.bl._stage_cost(_Sw, q)*(0.99**(k*0.03))
#             # cost = -ocp.bl._stage_cost(_Sw, q)*(1.0**k)
#             costs.append(cost.full()[0,0])
            
#             _Sw = f_discrete(_Sw, q, aw, bw[:,i]).full()[:,0]
#             new_Sw.extend(_Sw)
            
#         cost_traj.append(costs)
#         Sw = np.array(new_Sw)
        
#     return np.array(cost_traj)
        
    
# ## on true model
# aw_true = np.load("./data/a_true.npy")
# bw_true = np.load("./data/b_true.npy")

# Ne = 1
# nx = 25

# params_mpc = BLParamsSmpcShort(N_mpc = N_mpc, 
#                                 Ne = Ne,
#                                 nx = nx,
#                                 bw = bw_true[:,np.newaxis])

# qocp = np.load("./data/socp_qocp.npy")
# qmpc = [1.0]

# ocp = setup_sbl_ocp(params_mpc, 
#                     qmpc=qmpc,
#                     qocp=qocp)
    
# f_discrete = ocp.bl.simulate_at_k

# _Sw0 = [1.0]+[0.2]*(params_mpc.nx-1)
# Sw0 = np.array(_Sw0*Ne)  # initial state


# # Stochastic OCP
# qocp = np.load("./data/socp_qocp.npy")
# true_socp_cost = calculate_cost(f_discrete, Sw0, qocp, aw, bw_true[:,np.newaxis], Ne, nx)
# true_socp_cost_cum = np.cumsum(true_socp_cost, axis=0)

# # Stochastic MPC
# qsmpc = np.load("./data/smpc_qmpc.npy")
# true_smpc_cost = calculate_cost(f_discrete, Sw0, qsmpc, aw, bw_true[:,np.newaxis], Ne, nx)
# true_smpc_cost_cum = np.cumsum(true_smpc_cost, axis=0)

# # Stochastic MPC
# qsmpc_wse = np.load("./data/smpc_wse_qmpc.npy")
# true_smpc_wse_cost = calculate_cost(f_discrete, Sw0, qsmpc_wse, aw, bw_true[:,np.newaxis], Ne, nx)
# true_smpc_wse_cost_cum = np.cumsum(true_smpc_wse_cost, axis=0)

# # Stochastic MPC without terminal cost K_mpc = 5
# qsmpc_notc_5 = np.load("./data/smpc_notc_qmpc_5.npy")
# true_smpc_notc_cost_5 = calculate_cost(f_discrete, Sw0, qsmpc_notc_5, aw, bw_true[:,np.newaxis], Ne, nx)
# true_smpc_notc_cost_cum_5 = np.cumsum(true_smpc_notc_cost_5, axis=0)

# # Stochastic MPC without terminal cost K_mpc = 10
# qsmpc_notc_10 = np.load("./data/smpc_notc_qmpc_10.npy")
# true_smpc_notc_cost_10 = calculate_cost(f_discrete, Sw0, qsmpc_notc_10, aw, bw_true[:,np.newaxis], Ne, nx)
# true_smpc_notc_cost_cum_10 = np.cumsum(true_smpc_notc_cost_10, axis=0)

# # Stochastic MPC without terminal cost K_mpc = 20
# qsmpc_notc_20 = np.load("./data/smpc_notc_qmpc_20.npy")
# true_smpc_notc_cost_20 = calculate_cost(f_discrete, Sw0, qsmpc_notc_20, aw, bw_true[:,np.newaxis], Ne, nx)
# true_smpc_notc_cost_cum_20 = np.cumsum(true_smpc_notc_cost_20, axis=0)


# fig, ax = plt.subplots()
# fig.suptitle("Optimal control")
# stair_range = range(qocp.shape[0]-1)
# # ax.stairs(stair_range, qocp, linewidth=2, orientation="horizontal", baseline=None, label="SOCP")
# # ax.stairs(stair_range, qsmpc_wse, linewidth=2, orientation="horizontal", baseline=None, label="SMPC")
# ax.stairs(stair_range, qsmpc_notc_5, linewidth=2, orientation="horizontal", baseline=None, label="$K_{mpc} = 5$")
# ax.stairs(stair_range, qsmpc_notc_10, linewidth=2, orientation="horizontal", baseline=None, label="$K_{mpc} = 10$")
# ax.stairs(stair_range, qsmpc_notc_20, linewidth=2, orientation="horizontal", baseline=None, label="$K_{mpc} = 20$")
# fig.tight_layout()
# plt.legend()
# plt.grid(True)

# fig, ax = plt.subplots()
# fig.suptitle("NPV and cashflow profile on true model")
# # ax.plot(true_socp_cost, label="SOCP")
# # ax.plot(true_smpc_wse_cost, label="SMPC")
# ax.plot(true_smpc_notc_cost_5, label="$K_{mpc} = 5$")
# ax.plot(true_smpc_notc_cost_10, label="$K_{mpc} = 10$")
# ax.plot(true_smpc_notc_cost_20, label="$K_{mpc} = 20$")
# ax.legend(loc="center right")
# ax2 = ax.twinx()
# # ax2.plot(true_socp_cost_cum, linestyle="dashed") 
# # ax2.plot(true_smpc_wse_cost_cum, linestyle="dashed")
# ax2.plot(true_smpc_notc_cost_cum_5, linestyle="dashed")
# ax2.plot(true_smpc_notc_cost_cum_10, linestyle="dashed")
# ax2.plot(true_smpc_notc_cost_cum_20, linestyle="dashed")

# fig.tight_layout()
# plt.grid(True)
# plt.show()
# # plt.legend()

# socp_Sw_traj = np.load("./data/socp_Sw_traj.npy")
# smpc_Sw_traj = np.load("./data/smpc_Sw_traj.npy")
# smpc_wse_Sw_traj = np.load("./data/smpc_wse_Sw_traj.npy")
# smpc_notc_Sw_traj_5 = np.load("./data/smpc_notc_Sw_traj_5.npy")
# smpc_notc_Sw_traj_10 = np.load("./data/smpc_notc_Sw_traj_10.npy")
# smpc_notc_Sw_traj_20 = np.load("./data/smpc_notc_Sw_traj_20.npy")

# smpc_qwt, smpc_qot, smpc_swt, smpc_costt, smpc_fwt = get_flow_rate(socp_Sw_traj, Sw0, 10)
# smpc_wse_qwt, smpc_wse_qot, smpc_wse_swt, smpc_wse_costt, smpc_wse_fwt = get_flow_rate(smpc_wse_Sw_traj, smpc_wse_qt, 10)

# Nt = len(smpc_wse_qwt)
# plt.figure()
# plt.title("Fluid profile using SMPC")
# plt.plot(np.mean(smpc_wse_qwt, axis=1), label="water rate", color="blue")
# plt.fill_between(range(Nt), np.percentile(smpc_wse_qwt, 5, axis=1), np.percentile(smpc_wse_qwt, 95, axis=1),  color="blue", alpha=0.1)
# plt.plot(np.mean(smpc_wse_qot, axis=1), label="oil rate", color="green")
# plt.fill_between(range(Nt), np.percentile(smpc_wse_qot, 5, axis=1), np.percentile(smpc_wse_qot, 95, axis=1),  color="green", alpha=0.1)
# # plt.plot(fwt[:,:,0,0])
# # for i in range(Ne):
# #     plt.plot(smpc_swt[:,i], label=f"{i}-th ensemble")
# plt.grid()
# plt.legend(loc="center right")

# print(f"Cumulative cost at the last time step: \n",
#       f"SOCP: {true_socp_cost_cum[-1]} \n",
#       f"SMPC: {true_smpc_wse_cost_cum} \n",
#       f"K_mpc = 5: {true_smpc_notc_cost_cum_5} \n", 
#       f"K_mpc = 10: {true_smpc_notc_cost_cum_10} \n",
#       f"K_mpc = 20: {true_smpc_notc_cost_cum_20} \n")


## Comparison of SMPC with no terminal cost with different MPC window

In [8]:
# fig, ax = plt.subplots()
# fig.suptitle("Optimal control")
# stair_range = range(qocp[1:].shape[0]-1)
# ax.stairs(stair_range, qsmpc_notc_5[1:-1], linewidth=2, orientation="horizontal", baseline=None, label="$K_{mpc} = 5$")
# ax.stairs(stair_range, qsmpc_notc_10[1:-1], linewidth=2, orientation="horizontal", baseline=None, label="$K_{mpc} = 10$")
# ax.stairs(stair_range, qsmpc_notc_20[1:-1], linewidth=2, orientation="horizontal", baseline=None, label="$K_{mpc} = 20$")
# fig.tight_layout()
# ax.set_ylim(0.78, 1.22)
# plt.legend()
# plt.grid(True)
# plt.savefig("SMPC_no_terminal_cost_control.png")

# fig, ax = plt.subplots()
# fig.suptitle("NPV and cashflow profile on true model")
# ax.plot(true_smpc_notc_cost_5[:-1], label="$K_{mpc} = 5$")
# ax.plot(true_smpc_notc_cost_10[:-1], label="$K_{mpc} = 10$")
# ax.plot(true_smpc_notc_cost_20[:-1], label="$K_{mpc} = 20$")
# ax.legend()
# ax2 = ax.twinx()
# ax2.plot(true_smpc_notc_cost_cum_5, linestyle="dashed")
# ax2.plot(true_smpc_notc_cost_cum_10, linestyle="dashed")
# ax2.plot(true_smpc_notc_cost_cum_20, linestyle="dashed")

# fig.tight_layout()
# plt.grid(True)
# plt.savefig("SMPC_no_terminal_cost_cost.png")
# plt.show()
# print(f"Cumulative cost at the last time step: \n",
#       f"SOCP: {true_socp_cost_cum[-1]} \n",
#       f"SMPC: {true_smpc_wse_cost_cum[-2]} \n",
#       f"K_mpc = 5: {true_smpc_notc_cost_cum_5[-2]} \n", 
#       f"K_mpc = 10: {true_smpc_notc_cost_cum_10[-2]} \n",
#       f"K_mpc = 20: {true_smpc_notc_cost_cum_20[-2]} \n")



# ## Plot water and oil rate for all three cases
# fig, ax = plt.subplots(3, 1, figsize=(8,6))
# sw_traj_5 = np.load("./data/smpc_notc_Sw_traj_5.npy")
# sw_traj_10 = np.load("./data/smpc_notc_Sw_traj_10.npy")
# sw_traj_20 = np.load("./data/smpc_notc_Sw_traj_20.npy")


# Nt = sw_traj_5.shape[0]
# for _t in range(Nt):
      
#       if _t%10==0:
#             sw_flat_5 = sw_traj_5[_t,:]
#             sw_flat_10 = sw_traj_10[_t,:]
#             sw_flat_20 = sw_traj_20[_t,:]
            
#             for i in range(Ne):
#                   Si_5 = sw_flat_5[i*nx:(i+1)*nx]
#                   Si_10 = sw_flat_10[i*nx:(i+1)*nx]
#                   Si_20 = sw_flat_20[i*nx:(i+1)*nx]

#                   ax[0].plot(Si_5)
#                   ax[0].plot(Si_10)
#                   ax[0].plot(Si_20)
      
